In [1]:
import database as db
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import clear_output
from ipywidgets import HBox, VBox
from utils import search_by_index, search_by_pred, load_data, display_all
import traceback

dm = db.DataManager('default','./to_annotate/wu.txt')


args_type = ['arg0','arg1','arg2', 'arg3','arg4','loc','tmp','prp']
arg_list = []
pred_input = widgets.Text(description='predicate',layout=Layout(width='80%'))
for arg_str in args_type:
    arg_list.append(widgets.Text(description=arg_str,layout=Layout(width='80%')))
search_pred_input = widgets.Text(description='Pred search', layout=Layout(width="80%"))
search_index_input = widgets.Text(description='Index search', layout=Layout(width="80%"))
search_button = widgets.Button(description='Search')
left_box = VBox([pred_input]+arg_list[0:3],layout=Layout(width='30%'))
right_box = VBox(arg_list[3:],layout=Layout(width='30%'))
search_box = VBox([search_pred_input, search_index_input, search_button], layout=Layout(width="30%"))
box = HBox([left_box,right_box, search_box])


def submit_result(b):
    id_ = dm.fetch_current()
    try:
        arg_dict = {}
        pred_idx = int(pred_input.value)
        pred_input.value = ''
        for idx,t in enumerate(arg_list):
            if t.value != '':
                val = t.value.split(',')
                val = [int(i) for i in val]
                if len(val) == 1:
                    val.append(val[0])
                assert val[1] >= val[0]
                arg_dict[args_type[idx]] = val
                t.value =  ''
        id_ = dm.fetch_current()
        dm.save(id_,pred_idx,arg_dict)
        print('just commit: pred:',pred_idx,'arg:',arg_dict)
    except:
        print('error! please examine the input format')
        traceback.print_exc()

def next_sentence(b):
    dm.commit()
    id_ = dm.fetch_current()
    clear_output()
    load_data(dm,id_)
    display_all(dm, [box, left_btn_box], right_btn_box)
    
    
def out(b):
    db.output_annotations(dm.data)
    print('output done!')

def search(b):
    pred_value = search_pred_input.value
    index_value = search_index_input.value
    if index_value != '':
        if pred_value != '':
            print('If pred and index value are both assigned, search results are based on index value.')
        search_by_index(dm, index_value)
    elif pred_value != '':
        search_by_pred(dm, pred_value)
    else:
        print('error!!! You must assign the value of pred or index.')

def previous(b):
    __id = dm.fetch_prev()
    if __id == -1:
        print('There is no previous reocrd. It is the first one.')
    else:
        dm.data.sync()
        clear_output()
        load_data(dm, __id)
        display_all(dm, [box, left_btn_box], right_btn_box)

def next_page(b):
    __id = dm.fetch_next()
    clear_output()
    load_data(dm, __id)
    display_all(dm, [box, left_btn_box], right_btn_box)

def jump_by_index(b):
    index = int(jump_to.value)
    __id = dm.fetch_by_index(index)
    clear_output()
    load_data(dm, __id)
    display_all(dm, [box, left_btn_box], right_btn_box)
    
submit_button = widgets.Button(description="Pred submit",)
sent_submit_btn = widgets.Button(description='Sentence finish')
out_button = widgets.Button(description='Ouput result to json')

previous_btn = widgets.Button(description='Prev')
next_page_btn = widgets.Button(description='Next')
jump_to = widgets.Text(description='Jump to', layout=Layout(width='30%'))
jump_to_btn = widgets.Button(description='Jump')

left_btn_box = HBox([submit_button, sent_submit_btn, out_button])
right_btn_box = HBox([previous_btn, next_page_btn, jump_to, jump_to_btn])

# button_box = VBox([left_btn_box, right_btn_box])

out_button.on_click(out)
sent_submit_btn.on_click(next_sentence)
submit_button.on_click(submit_result)
search_button.on_click(search)
previous_btn.on_click(previous)
next_page_btn.on_click(next_page)
jump_to_btn.on_click(jump_by_index)


id_ = dm.fetch_next()
load_data(dm,id_)
display_all(dm, [box, left_btn_box], right_btn_box)

possible predicate:
['有(1) ', '关注(2) ', '没有(7) ', '主演(15) ', '获得(22) ', '获奖(30) ', '有(39) ', '去(41) ', '关注(42) ', '还有(48) ', '主演(50) ', '为(56) ', '主演(62) ', '是(63) ', '主演(74) ', '有(81) ', '关注(85) ']
Done labels:
pred: 有(1)	arg0: 你(-2)	arg1: 关注什么电影吗(2,5)	tmp: 最近(0,0)	
pred: 关注(2)	arg0: 你(-2)	arg1: 电影(4,4)	tmp: 最近(0,0)	
pred: 没有(7)	arg0: 我(-1)	arg1: 关注什么电影(2,4)	
pred: 主演(15)	arg1: 僵尸新娘(17,17)	arg3: 由海伦娜·伯翰·卡特(9,14)	
pred: 获得(22)	arg0: 僵尸新娘(17,17)	arg1: 安妮奖(2006；第33届)获奖乌布·伊沃克斯技术成就奖(24,35)	
pred: 关注(42)	arg0: 我(38,38)	arg1: 僵尸新娘(17,17)	tmp: 有时间(39,40)	
pred: 去(41)	arg0: 我(38,38)	arg1: 关注一下(42,43)	
pred: 去(41)	arg0: 我(38,38)	arg1: 关注一下(42,43)	tmp: 有时间(39,40)	
pred: 还有(48)	arg1: 一位主演我很喜欢(49,51)	loc: 这部电影里面(45,47)	
pred: 为(56)	arg0: 他(53,53)	arg4: 第77届奥斯卡奖最佳男主角(57,59)	
pred: 是(63)	arg0: 主演(62,62)	arg1: 谁(64,64)	
pred: 主演(74)	arg0: 约翰尼·德普(67,69)	arg1: 不一样的天空(71,71)	
pred: 关注(85)	arg0: 我(83,83)	arg1: 约翰尼·德普(67,69)	tmp: 有时间(81,82)	


progress:21/9232
